## AutoGen

AutoGen is an open-source programming framework for building AI agents and facilitating cooperation among multiple agents to solve tasks. 

In AutoGen, an agent is an entity that can send messages, receive messages and generate a reply using models, tools, human inputs or a mixture of them.

![Agent](https://microsoft.github.io/autogen/assets/images/autogen_agents-b80434bcb15d46da0c6cbeed28115f38.png)

### Company Information Search Application

In this example, let us use an AutoGen agent system to get information for a given company. It involves two steps.

- Given the name of the company, search the internet and get the URL of the company.
- Extract the content of the URL and then summarize the content about the company.

In [1]:
import os
import requests
from bs4 import BeautifulSoup
from typing import Annotated, Literal
from duckduckgo_search import DDGS
from autogen import ConversableAgent

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Setting up the LLM config
llm_config = {
    "config_list": [{"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]}]
}

First let us create an agent that has a tool to search the internet and gets the URL of the company.

In [3]:
url_assistant = ConversableAgent(
    name="URL Assistant",
    system_message="You are a URL fetcher assistant. "
    "You can help with retrieving the website url of the given company name using search. "
    "Give the company website base url in json format - {url: <url>}. "
    "Add 'TERMINATE' when the task is done.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [4]:
# A tool to help get the information of the company using duckduckgo search
def company_search(company_name: str) -> str:
    results = DDGS().text(f"Website of {company_name}", max_results=5)
    return str(results)

Now let us create another agent that summarizes the website content

In [5]:
summary_assistant = ConversableAgent(
    name="Summary Assistant",
    system_message="You are a Summary assistant. "
    "You help with getting the content from the website and summarizing it. "
    "Get the web page text and summarize it in one paragraph. "
    "Add 'TERMINATE' when the task is done.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [6]:
# Tool to extract the content of the webpage
def get_webpage_text(url: str) -> str:
    try:
        # Fetch the content of the webpage
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Get the text from the parsed HTML
        text = soup.get_text(separator=' ', strip=True)
        
        return text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the webpage: {e}")
        return None

We will also have a flow agent that just manages the flow and executes the tool

In [7]:
flow_agent = ConversableAgent(
    name="Flow Agent",
    system_message="You give me the text that I give you.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

Now let us register the tool (functions) that we created to the corresponding assistant.

In [8]:
from autogen import register_function

register_function(
    company_search,
    caller=url_assistant,
    executor=flow_agent,
    name="company_search",
    description="Searches information about the given company",
)

# Register the function to the agent.
register_function(
    get_webpage_text,
    caller=summary_assistant, 
    executor=flow_agent, 
    name="get_webpage_text", 
    description="Extracts the content of the given web page url",  # A description of the tool.
)

In [9]:
company_name = "Deeplearning AI"

chat_results = flow_agent.initiate_chats(
    [
        {
            "recipient": url_assistant,
            "message": company_name,
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": summary_assistant,
            "message": "",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
Flow Agent (to URL Assistant):

Deeplearning AI

--------------------------------------------------------------------------------
URL Assistant (to Flow Agent):

***** Suggested tool call (call_wEqcgPFe64vepkHOaXIMgoVq): company_search *****
Arguments: 
{"company_name":"Deeplearning AI"}
*******************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION company_search...
Flow Agent (to URL Assistant):

Flow Agent (to URL Assistant):

***** Response from calling tool (call_wEqcgPFe64vepkHOaXIMgoVq) *****
[{'title': 'DeepLearning.AI: Start or Advance Your Career in AI', 'href': 'https://www.deeplearning.ai/', 'body': 'Llama 3.1 is State-of-the-Art and Open, Web Data Goes Dark, Open

In [10]:
chat_results[-1].summary

'DeepLearning.AI provides courses and resources for individuals to start or advance their careers in AI. The platform offers AI Python for Beginners course, AI newsletter, blog, forum, and events. Andrew Ng, a prominent figure in the AI field, shares insights and news on AI developments. Users can enroll in AI courses and specializations to build machine learning and AI skills for real-world applications. The platform also offers free resources such as practical guides on AI and machine learning, career development in AI, and books like Machine Learning Yearning and A Complete Guide to Natural Language Processing. Subscribe to stay updated on the latest AI news, events, and courses. Users can join a community of over 7 million people learning AI and explore career opportunities in AI. '